# Linear Algebra

## Chapter 1: Numerical vector space

$x \in A$: x is an **ELEMENT** of A  
$A \subseteq B$: A is **SUBSET** of B  
$\varnothing, \emptyset$: Empty set  
$A \cap B$: Intersection  
$A \cup B$: Union  
$A^c, \overline{A} = \{x \in U \mid x \notin A\}$: Complement  
$A \setminus B, A - B = \{x \mid x \in A \cap x \notin B\}$: Difference  
$\forall x$: All x  
$\exists x$: A x  

### Vector Class

In [ ]:
class vector(list):
    
    def __init__(self, __content__):
        if not isinstance(__content__, list):
            raise TypeError('入力はlist型のみでよろしく')
        list.__init__(self, __content__)
    
    
    def broadcast(func):
        def inner(self, other):
            if isinstance(other, (int, float)):
                other = [other] * len(self)
            if isinstance(other, list):
                if len(self) != len(other):
                    raise ArithmeticError('あれ？サイズ違うっぽくない？')
                return func(self, other)
            raise TypeError('なんか思ってたんとちゃう...!!!')
        return inner
    
    
    # 足し算
    @broadcast
    def __add__(self, other):
        return vector([x + y for x, y in zip(self, other)])
    
    def __radd__(self, other):
        return self.__add__(other)
    
    
    # 引き算
    @broadcast
    def __sub__(self, other):
        return vector([x - y for x, y in zip(self, other)])
    
    @broadcast
    def __rsub__(self, other):
        return vector([x - y for x, y in zip(other, self)])
    
    
    # 掛け算
    @broadcast
    def __mul__(self, other):
        return vector([x * y for x, y in zip(self, other)])
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    
    # 割り算
    @broadcast
    def __truediv__(self, other):
        return vector([x / y for x, y in zip(self, other)])
    
    @broadcast
    def __rtruediv__(self, other):
        return vector([x / y for x, y in zip(other, self)])
    
    
    # 累乗
    @broadcast
    def __pow__(self, other):
        return vector([x ** y for x, y in zip(self, other)])
    
    
    # 内積
    def __matmul__(self, other):
        if isinstance(other, matrix):
            return matrix([self]) @ other
        return sum(self.__mul__(other))
    
    def __rmatmul__(self, other):
        return sum(self.__mul__(other))
    
    
    def __str__(self):
        return '[%s]' % (' '.join(map(str, self)))
    
    
    def __repr__(self):
        return '%s(%s)' % (self.__class__.__name__, super().__repr__())

### Matrix Class

In [ ]:
class MatrixShapeError(Exception):
    pass

In [ ]:
class matrix(list):
    
    @staticmethod
    def zeros(n):
        if isinstance(n, int):
            return vector([0] * n)
        if isinstance(n, tuple) and len(n) == 2:
            return matrix([[0] * n[1]  for _ in range(n[0])])
    
    @staticmethod
    def eye(n, m=None):
        if isinstance(m, int):
            return matrix([[int(ni == mi) for mi in range(m)] for ni in range(n)])
        return matrix([[int(ni == mi) for mi in range(n)] for ni in range(n)])
    
    def __init__(self, __content__):
        # 入力がlist型になっているかどうか判定
        if not isinstance(__content__, list):
            raise TypeError('入力はlist型のみでよろしく')
        
        # 入力が2次元配列になっているか判定
        self.n = len(__content__)
        if self.n == 0 or isinstance(__content__[0], (int, float)):
            raise MatrixShapeError('ねぇキミ、Matrixの意味は知ってる？')
        
        # 入力の形がn * m行列になっているか判定
        self.m = len(__content__[0])
        for row in __content__:
            if not len(row) == self.m:
                raise MatrixShapeError('その行列、いびつなんだけど...!')
        
        list.__init__(self, map(vector, __content__))
    
    
    def broadcast(func):
        def inner(self, other):
            if isinstance(other, (int, float)):
                other = [[other] * len(self[0])] * len(self)
            if isinstance(other, list):
                if len(self) != len(other):
                    raise ArithmeticError('あれ？サイズ違うっぽくない？')
                return func(self, other)
            raise TypeError('なんか思ってたんとちゃう...!!!')
        return inner
    
    
    # 足し算
    @broadcast
    def __add__(self, other):
        return matrix([x + y for x, y in zip(self, other)])
    
    def __radd__(self, other):
        return self.__add__(other)
    
    
    # 引き算
    @broadcast
    def __sub__(self, other):
        return matrix([x - y for x, y in zip(self, other)])
    
    @broadcast
    def __rsub__(self, other):
        return matrix([x - y for x, y in zip(other, self)])
    
    
    # 掛け算
    @broadcast
    def __mul__(self, other):
        return matrix([x * y for x, y in zip(self, other)])
    
    def __rmul__(self, other):
        return self.__mul__(other)
    
    
    # 割り算
    @broadcast
    def __truediv__(self, other):
        return matrix([x / y for x, y in zip(self, other)])
    
    @broadcast
    def __rtruediv__(self, other):
        return matrix([x / y for x, y in zip(other, self)])
    
    
    # 累乗
    @broadcast
    def __pow__(self, other):
        return matrix([x ** y for x, y in zip(self, other)])
    
    
    # 行列積
    def __matmul__(self, other):
        if isinstance(other, matrix):
            if self.m == other.n:
                other = other.transpose()
                return matrix([[o @ s for o in other] for s in self])
            else:
                raise ArithmeticError('行列のサイズが違うんだけど')
        raise TypeError('行列としか計算できないよ')
    
    
    def row(self, n):
        return vector(self[n])
    
    def col(self, n):
        return vector([vec[n] for vec in self])
    
    def transpose(self):
        return matrix([[vec[i] for vec in self] for i in range(self.m)])
    
    def sliceitem(self, r, c):
        r = slice(r, r + 1, None) if isinstance(r, int) else r
        c = slice(c, c + 1, None) if isinstance(c, int) else c
        disp = matrix([self.row(ri) for ri in range(self.n)[r]])
        disp = matrix([disp.col(ci) for ci in range(self.m)[c]])
        return disp.transpose()
    
    # 要素の取得
    def __getitem__(self, idx):
        if isinstance(idx, (int, slice)):
            return vector(self.disp()[idx])
        if isinstance(idx, tuple) and len(idx) == 2:
            i, j = idx
            if isinstance(i, int) and isinstance(j, int):
                return self.disp()[i][j]
            return self.sliceitem(i, j)
        raise IndexError('ちょっとインデックス多い気がするんだけど')
    
    def disp(self):
        return [list(x) for x in self]
    
    def __str__(self):
        return '%s' % self.disp()
    
    def __repr__(self):
        return '%s(%s)' % (self.__class__.__name__, self.disp())

### Operation Checking

#### Vector Calculation

In [ ]:
A = vector([1, 2, 3])
B = vector([3, 3, 2])

In [ ]:
A + 10 + 5

In [ ]:
10 + (5 + A)

In [ ]:
A - 1 - 2

In [ ]:
10 - (3 - A)

In [ ]:
A * 2 * 1

In [ ]:
1 * (2 * A)

In [ ]:
A / 2 / 1

In [ ]:
15 / (3 / A)

In [ ]:
A + B

In [ ]:
A - B

In [ ]:
B - A

In [ ]:
A * B

In [ ]:
A / B

In [ ]:
B / A

In [ ]:
A @ B

In [ ]:
A @ B[::-1]

#### Matrix Calculation

In [ ]:
A = matrix([[1, 2], [3, 4]])
B = matrix([[5, 6], [7, 8]])

In [ ]:
2 + A + 1

In [ ]:
3 - A - 1

In [ ]:
1 * A * 2

In [ ]:
A / 2

In [ ]:
12 / A

In [ ]:
A + B

In [ ]:
A - B

In [ ]:
B - A

In [ ]:
A * B

In [ ]:
A / B

In [ ]:
B / A

In [ ]:
A @ B

In [ ]:
B @ A

In [ ]:
A.transpose()

In [ ]:
matrix.zeros(10)

In [ ]:
matrix.zeros((3, 4))

In [ ]:
matrix.eye(3)

In [ ]:
matrix.eye(2,5)

## Chapter2: Sweep Method

In [ ]:
import numpy as np

In [ ]:
A = np.array([
    [0, -1, -2, 2, 5],
    [2, -1, -3, 3, 10],
    [-1, 3, 3, -2, 2],
    [1, 2, 0, -1, -10]
], dtype=float)

In [ ]:
def sweep(A):
    n = len(A)
    
    for i in range(n):
        # 部分ピボット選択
        A = A[np.concatenate([np.array(range(i), dtype=int), np.argsort(np.abs(A[i:,i]))[::-1] + i])]
        
        pivot = A[i, i]
        
        if pivot == 0:
            raise ZeroDivisionError('ちょ、おまｗｗｗ0で割ってるｗｗｗ')
        
        A[i] = A[i] / pivot
        
        for j in range(i + 1, n):
            coef = A[j, i]
            A[j] -=  A[i] * coef
    
    for i in range(n)[::-1]:
        for j in range(i):
            coef = A[j, i]
            A[j] -= A[i] * coef
    
    return A

In [ ]:
sweep(A)